# System Design Examples: Rate Limiter, Cache, and URL Shortener
This notebook contains Python implementations of three classic system design interview problems:
- Rate Limiter
- Cache (LRU)
- URL Shortener

## 1. Rate Limiter (Token Bucket / Sliding Window)
This example uses a simple **fixed window counter** approach to limit requests per user.

In [ ]:
import time
from collections import defaultdict


class RateLimiter:
    def __init__(self, max_requests, window_seconds):
        self.max_requests = max_requests
        self.window_seconds = window_seconds
        self.user_access = defaultdict(list)

    def is_allowed(self, user_id):
        current_time = time.time()
        window_start = current_time - self.window_seconds

        # Remove expired timestamps
        self.user_access[user_id] = [
            timestamp
            for timestamp in self.user_access[user_id]
            if timestamp > window_start
        ]

        if len(self.user_access[user_id]) < self.max_requests:
            self.user_access[user_id].append(current_time)
            return True
        return False


# Usage
limiter = RateLimiter(max_requests=3, window_seconds=10)
for i in range(5):
    print("Request allowed:", limiter.is_allowed("user1"))
    time.sleep(2)

Request allowed: True
Request allowed: True
Request allowed: True
Request allowed: False
Request allowed: False


## 2. Cache - Least Recently Used (LRU)
Python’s `OrderedDict` can be used to implement an efficient LRU cache.

In [ ]:
from collections import OrderedDict


class LRUCache:
    def __init__(self, capacity):
        self.cache = OrderedDict()
        self.capacity = capacity

    def get(self, key):
        if key not in self.cache:
            return -1
        self.cache.move_to_end(key)
        return self.cache[key]

    def put(self, key, value):
        if key in self.cache:
            self.cache.move_to_end(key)
        self.cache[key] = value
        if len(self.cache) > self.capacity:
            self.cache.popitem(last=False)


# Usage
cache = LRUCache(2)
cache.put("a", 1)
cache.put("b", 2)
print("Get a:", cache.get("a"))  # Access 'a', now 'b' is LRU
cache.put("c", 3)  # 'b' should be evicted
print("Get b:", cache.get("b"))

Get a: 1
Get b: -1


## 3. URL Shortener
Simulate a service like bit.ly using a base62-encoded ID.

In [ ]:
import string
import random


class URLShortener:
    def __init__(self):
        self.url_to_code = {}
        self.code_to_url = {}
        self.base = string.ascii_letters + string.digits
        self.prefix = "https://short.ly/"

    def encode(self, long_url):
        if long_url in self.url_to_code:
            return self.prefix + self.url_to_code[long_url]

        code = "".join(random.choices(self.base, k=6))
        while code in self.code_to_url:
            code = "".join(random.choices(self.base, k=6))

        self.url_to_code[long_url] = code
        self.code_to_url[code] = long_url
        return self.prefix + code

    def decode(self, short_url):
        code = short_url.split("/")[-1]
        return self.code_to_url.get(code, None)


# Usage
shortener = URLShortener()
short = shortener.encode("https://example.com/very/long/url")
print("Short URL:", short)
print("Decoded:", shortener.decode(short))

Short URL: https://short.ly/iK7Wui
Decoded: https://example.com/very/long/url
